In [1]:
import requests
import json
import pandas as pd

In [2]:
filepath = '/Users/jacksonpeoples/Desktop/Springboard/CapstoneTwo/Data/Vision_Zero_Crash_Report_Data.csv'
crash_data = pd.read_csv(filepath)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Checking out some basic info on the dataset:
crash_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161824 entries, 0 to 161823
Data columns (total 51 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   crash_id                            161824 non-null  int64  
 1   crash_fatal_fl                      161824 non-null  object 
 2   crash_date                          161824 non-null  object 
 3   crash_time                          161824 non-null  object 
 4   case_id                             159592 non-null  object 
 5   rpt_latitude                        5349 non-null    float64
 6   rpt_longitude                       5349 non-null    float64
 7   rpt_block_num                       140411 non-null  object 
 8   rpt_street_pfx                      90518 non-null   object 
 9   rpt_street_name                     161818 non-null  object 
 10  rpt_street_sfx                      118702 non-null  object 
 11  crash_speed_limit         

In [4]:
#Quite a few columns. Many will be useful, some won't. Some have very few values and some convey the same information
#in a different form.
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,case_id,rpt_latitude,rpt_longitude,rpt_block_num,rpt_street_pfx,rpt_street_name,...,bicycle_death_count,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl
0,16776868,N,12/04/2018 03:36:00 PM,15:36:00,183381097,NaN,NaN,1900,N,IH 35 SB,...,0,0,0,0,0,0,0,0,Y,N
1,16786471,N,12/07/2018 01:38:00 PM,13:38:00,183410837,NaN,NaN,12400,NaN,N IH 35 SVRD SB,...,0,0,0,0,0,0,0,0,Y,N
2,16757244,N,11/18/2018 04:10:00 AM,04:10:00,183220392,NaN,NaN,4500,N,N IH 35 NB,...,0,0,0,0,0,0,0,0,Y,N
3,16776852,N,12/03/2018 11:19:00 AM,11:19:00,183370654,NaN,NaN,200,NaN,NUECES ST,...,0,0,0,0,0,0,0,0,N,N
4,16765750,N,11/29/2018 09:07:00 AM,09:07:00,183330494,NaN,NaN,4100,N,LAMAR,...,0,0,0,0,0,0,0,0,Y,N


In [5]:
#Will go ahead and get rid of columns that pertain to police response (not helpful in predicting danger) and
#a few that are almost entirely empty.
del_col = ['case_id', 'rpt_latitude', 'rpt_longitude', 'contrib_factr_p1_id', 'contrib_factr_p2_id', 'units_involved','atd_mode_category_metadata', 'street_nbr_2']
crash_data = crash_data.drop(del_col, 1)

In [6]:
#Another question worth asking is should I regard fatality info? Use it to weight the danger of a given crash,
#or should I drop the data and focus only on the likelihood of a crash period?
crash_data['crash_fatal_fl'].value_counts()

N    161035
Y       789
Name: crash_fatal_fl, dtype: int64

In [8]:
sample = crash_data['crash_date'][7]
print(sample)
print(type(sample))

11/21/2018 05:39:00 PM
<class 'str'>


In [11]:
#Would like to get rid of the time info in 'crash_date' so that I can work with date/time of day
#independently.
def split_time(crash_date):
    tup = crash_date.partition(' ')
    return tup[0]
print(split_time(sample))

11/21/2018


In [12]:
#Will now apply the function to the 'crash_date' column:
crash_data['crash_date'] = crash_data['crash_date'].apply(split_time)

In [13]:
#See if it worked:
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,rpt_block_num,rpt_street_pfx,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,...,bicycle_death_count,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl
0,16776868,N,12/04/2018,15:36:00,1900,N,IH 35 SB,NaN,-1.0,N,...,0,0,0,0,0,0,0,0,Y,N
1,16786471,N,12/07/2018,13:38:00,12400,NaN,N IH 35 SVRD SB,NaN,70.0,N,...,0,0,0,0,0,0,0,0,Y,N
2,16757244,N,11/18/2018,04:10:00,4500,N,N IH 35 NB,NaN,70.0,N,...,0,0,0,0,0,0,0,0,Y,N
3,16776852,N,12/03/2018,11:19:00,200,NaN,NUECES ST,NaN,30.0,N,...,0,0,0,0,0,0,0,0,N,N
4,16765750,N,11/29/2018,09:07:00,4100,N,LAMAR,BLVD,-1.0,N,...,0,0,0,0,0,0,0,0,Y,N


In [17]:
#Need to convert both 'crash_date' and 'crash_time' to timeseries types.
crash_data['crash_date']= pd.to_datetime(crash_data['crash_date'])
crash_data['crash_time']= pd.to_datetime(crash_data['crash_time'])
crash_data['crash_time'] = [time.time() for time in crash_data['crash_time']]

In [18]:
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,rpt_block_num,rpt_street_pfx,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,...,bicycle_death_count,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl
0,16776868,N,2018-12-04,15:36:00,1900,N,IH 35 SB,NaN,-1.0,N,...,0,0,0,0,0,0,0,0,Y,N
1,16786471,N,2018-12-07,13:38:00,12400,NaN,N IH 35 SVRD SB,NaN,70.0,N,...,0,0,0,0,0,0,0,0,Y,N
2,16757244,N,2018-11-18,04:10:00,4500,N,N IH 35 NB,NaN,70.0,N,...,0,0,0,0,0,0,0,0,Y,N
3,16776852,N,2018-12-03,11:19:00,200,NaN,NUECES ST,NaN,30.0,N,...,0,0,0,0,0,0,0,0,N,N
4,16765750,N,2018-11-29,09:07:00,4100,N,LAMAR,BLVD,-1.0,N,...,0,0,0,0,0,0,0,0,Y,N


In [19]:
#create a function to get the weekday for each row based on the date.
def get_weekday(date):
    return date.weekday()

#use that function to create weekday column.
crash_data['Weekday'] = crash_data['crash_date'].apply(get_weekday)

In [20]:
crash_data['Weekday'].value_counts()

4    26897
3    24202
1    23605
2    23297
5    22844
0    21798
6    19181
Name: Weekday, dtype: int64

In [21]:
crash_data['crash_date'].value_counts()

2014-01-08    111
2017-12-06    104
2018-12-06    102
2012-12-31    100
2017-03-11     98
             ... 
2018-12-25     11
2010-01-10     10
2017-12-25      9
2010-02-24      8
2011-09-05      5
Name: crash_date, Length: 3896, dtype: int64

In [25]:
print(crash_data['crash_date'].max())
print(crash_data['crash_date'].min())

2020-08-31 00:00:00
2010-01-01 00:00:00
